In [67]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from src import parse
from collections import Counter
import random

import logging
import sys


log = logging.getLogger()
log.setLevel(logging.DEBUG)

if not log.hasHandlers():
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    log.addHandler(handler)

/Users/andrew/src/otp-scheduler


In [68]:
# ASSUMPTIONS
MEAL_CAPACITY = 20000
MAX_DROPOFFS_PER_RUN = 4
MIN_MEALS_PER_RUN = 70 # not used, though

# This can be set a value from 0 (no effect) to the total count of restaurants (large effect)
RESTAURANT_FOUNDER_KNOB = 15
# Sometimes you may need to fine tune the meal steal knob above if one step is too big of a leap.
# This throws in randomness (0-1) for how often we trigger the meal steal logic (whether or not there is a meal steal opportunity)
RESTAURANT_FOUNDER_RANDOM_KNOB = 0.5

# Hard to describe, but a higher-priority restaurant can't leapfrog the current candidate restaurant unless it has
# served this hospital fewer times than the current candidate restaurant, with an "epsilon" of this value.
RESTAURANT_DIVERSITY_DIFF_KNOB = 5
DIVERSITY_KNOB = 1

curr_capacity = MEAL_CAPACITY

def sort_schedule(df):
    return df.sort_values(by=['zone', 'priority_rank', 'code']).reset_index(drop=True)

restaurants, num_zones = parse.parse_from_restaurants_csv('../../examples/boston/20200622/restaurants.csv')
restaurants.set_index('code', inplace=True)
hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200622/hospitals.csv')).set_index('name')
vip_hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200622/vips.csv')).set_index('name')
vip_schedule = vip_hospitals.iloc[:, 4:]

assert hospitals['zone'].max() <= num_zones, 'A hospital is in a zone that we didn\'t expect from the restaurant sheet!'

rest_usage = restaurants.copy()
rest_usage.iloc[:, 3:(rest_usage.shape[1] - num_zones)] = 0

rest_cap = restaurants.copy()

total_hospitals = hospitals.shape[0]

print('Number of Restaurants:', restaurants.shape[0])
print('Founding (Priority <= 1) Restaurants:', restaurants[restaurants['priority'] <= 1].index.values)
print('Number of Hospitals:', total_hospitals)
print('Number of Zones:', num_zones)

Number of Restaurants: 28
Founding (Priority <= 1) Restaurants: ['PG' 'LD' 'MM' 'CS' 'MD' 'ME' 'RT' 'CE' 'FV' 'MY' 'CL' 'FC' 'JC' 'TB'
 'IK' 'RO' 'PM' 'LP' 'PS' 'TK' 'HH']
Number of Hospitals: 42
Number of Zones: 7


In [69]:
def day_from_meal(meal):
    return meal.replace('lunch', '').replace('breakfast', '').replace('dinner', '').replace('_', '')

# Maybe not needed
day_adjacency = {
    'mon': ['tues', 'wed'],
    'tues': ['mon', 'wed'],
    'wed': ['tues', 'thurs'],
    'thurs': ['wed', 'fri'],
    'fri': ['thurs', 'sat'],
    'sat': ['fri', 'sun'],
    'sun': ['fri', 'sat']
}

In [70]:
meal_log = None
meal_log_key_kludge = set()

def get_meal_log(schedule, is_vip):
    log = []
    for name, row in schedule.iterrows():
        for j, val in enumerate(row[4:]):
            # Add only if this cell is not blank, AND we haven't already added this hospital/meal.
            # This is possible if this order exists in both the regular and VIP meal schedule.
            # Since we add the VIP meals first, then we're guaranteed for it to be counted as a VIP meal if it's in both places.
            if not pd.isnull(val) and (name, schedule.columns[j + 4]) not in meal_log_key_kludge:
                log.append([
                    name,
                    row['zone'],
                    row['priority_rank'],
                    is_vip,
                    schedule.columns[j + 4],
                    val, 
                    np.nan,
                    day_from_meal(schedule.columns[j + 4]),
                    False
                ])
                meal_log_key_kludge.add((name, schedule.columns[j + 4]))
    return pd.DataFrame(log, columns=['hospital', 'zone', 'priority_rank', 'vip', 'meal', 'quantity', 'restaurant', 'day', 'failed'])

vip_log = get_meal_log(vip_hospitals, True)
meal_log = get_meal_log(hospitals, False).append(vip_log)

In [71]:
log.info(meal_log.shape)

INFO - (297, 9)


In [72]:
def get_rest_cap_cluster_for_meal(meal, zone):
    z = 'zone_' + str(zone)
    r = rest_cap[rest_cap[z] > 0][['priority', meal, z]]
    return r

In [73]:
def get_oustanding_vip_meals():
    return meal_log[(meal_log['vip'] == True) & meal_log['restaurant'].isnull()]

def get_outstanding_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone) & meal_log['restaurant'].isnull()]

def get_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone)]

def is_restaurant_serving_same_day_other_meal(rest_code, meal):
    day = day_from_meal(meal)
    rest_serving = meal_log[(meal_log['restaurant'] == rest_code) & (meal_log['day'] == day) & meal_log['meal'] != meal]
    return not rest_serving.empty

def assign_hospital_order(row):
    global curr_capacity

    meal = row['meal']
    hospital = row['hospital']
    zone = row['zone']
    
    print('Assigning', hospital, meal)
    
    # Check to see if we've already done this (sanity)
    if not pd.isnull(row['restaurant']):
        print('We have already assigned this one!')
        return False
        
    hospitals_in_cluster = get_hospitals_in_cluster_for_meal(meal, zone)

    # Restaurant capacity
    r_cap = get_rest_cap_cluster_for_meal(meal, zone)
        
    # The amount have to serve
    target_quantity = row['quantity']
    
    if curr_capacity - target_quantity < 0:
        print('We cannot to afford this meal because of our global limit!', curr_capacity, target_quantity)
        return False
    
    print(hospital, meal, zone, target_quantity)
    
    def find_restaurant():
        # Check if a restaurant is already serving the cluster
        restaurants_serving_cluster = hospitals_in_cluster[hospitals_in_cluster['restaurant'].notnull()]['restaurant']
        
        if not restaurants_serving_cluster.empty:
            # Filter out any that are already at the dropout limit
            rest_counter = Counter(restaurants_serving_cluster)
            rest_cluster_keys = rest_counter.keys()
            print('Checking for restaurants already in cluster', rest_counter)
            rest_with_room = [c[0] for c in rest_counter.most_common() if c[1] < MAX_DROPOFFS_PER_RUN]
            
            # Find restaurants with capacity
            candidates_with_quantity = []
            for r in rest_with_room:
                q = rest_cap.loc[r, meal]
                if q >= target_quantity:
                    print('Found an existing restaurant with capacity', r, q)
                    candidates_with_quantity.append(r)
                    if is_restaurant_serving_same_day_other_meal(r, meal):
                        print('Returning a restaurant with capacity, working on the same day', r, q)
                        return r
            if len(candidates_with_quantity):
                print('Returning a restaurant with capacity', r, q)
                return candidates_with_quantity[0]
        
        # Couldn't find an appropriate restaurant already serving cluster
        print('No suitable restaurants already working this cluster')
        
        # Find restaurants that are already serving the same meal time but in a different zone
        restaurants_in_other_zones = meal_log[
            (meal_log['meal'] == meal) & (
                meal_log['zone'] != zone) & (
                meal_log['restaurant'].notnull())]['restaurant'].unique()
                
        print('filtering down candidates with three lists', restaurants_serving_cluster, restaurants_in_other_zones)
        r_candidates = r_cap[
            (~r_cap.index.isin(restaurants_serving_cluster)) & (
                ~r_cap.index.isin(restaurants_in_other_zones)) & (
                r_cap[meal] >= target_quantity)
        ]
        
        print('Number of candidates with capacity', r_candidates.shape[0])
        
        if r_candidates.empty:
            return None
        
        # TODO add is_restaurant_serving_same_day_other_meal logic down here too!
        
        zone_col = 'zone_' + str(zone)

        # Sort in zone preference + (maybe) priority + quantity order (TODO KNOB: QUANTITY VAL USED TO BE ascending=False)
        r_candidates = r_candidates.sort_values(by=[zone_col, meal], ascending=[False, True])
        
        day = day_from_meal(meal)
        
        # TODO Make sure restaurant isn't overbooked on their max day count
        final_valid_list = [] # tuple of rest_code, priority, meals already served for this hospital
        for i, r in r_candidates.iterrows():
            print(i)
            days_worked = set(meal_log[meal_log['restaurant'] == i]['day'])
            # Check if we're already working this day, or the new day is still under cap
            if day in days_worked or len(days_worked) < rest_cap.loc[i, 'max_days_per_week']:
#                 return i
                final_valid_list.append([i, r['priority'], meal_log[(meal_log['hospital'] == hospital) & (meal_log['restaurant'] == i)].shape[0]])
        
        # MEAL STEAL -- We need to prioritize founding Restaurants (like "PG, LD" in the Boston case)
        # TODO -- parameterize this, both the restaurants we want to do this for, and also how much we crank this knob
        # Comment out this section to ignore
        if len(final_valid_list) == 0:
            return None
        else:
            if len(final_valid_list) == 1:
                return final_valid_list[0][0]
            else:
                # First, sort by times the restaurant has served this hospital (ascending)
                # Variety is the spice of life, after all, and we need to keep these hospital workers excited
                # for a new day!
                # TODO we may have to add a random knob here
#                 final_valid_list = sorted(final_valid_list[0:DIVERSITY_KNOB], key=lambda x: x[2])
                if RESTAURANT_FOUNDER_RANDOM_KNOB > 0 and random.random() <= RESTAURANT_FOUNDER_RANDOM_KNOB:
                    print('Entering potential meal steal', final_valid_list[0:RESTAURANT_FOUNDER_KNOB])
#                     for r in final_valid_list[0:RESTAURANT_FOUNDER_KNOB]:
#                         if r in founding_restaurants:
#                             print('Meal Steal happened!', r)
#                             return r
                    original_selection = final_valid_list[0]
#                     find_highest_pri = sorted(final_valid_list[0:RESTAURANT_FOUNDER_KNOB], key=lambda x: x[1])
                    
                    def sorter(x):
                        # Just return orig priority for the first one
                        if x[0] == original_selection[0]:
                            return x[1]
                        # if priority is better AND we're within one meal served of the original, replace it
                        if x[1] < original_selection[1] and x[2] <= original_selection[2] + RESTAURANT_DIVERSITY_DIFF_KNOB:
                            return x[1]
                        # Send it to the back
                        return 1000000
            
                    find_highest_pri = sorted(final_valid_list[0:RESTAURANT_FOUNDER_KNOB], key=sorter)
                    print('Sorted by priority / meal diversity', find_highest_pri)
                    return find_highest_pri[0][0]
                return final_valid_list[0][0]
        
#         return None
          
    r_code = find_restaurant()
    
    if r_code is None:
        print('CANNOT SERVE HOSPITAL FOR THEIR REQUEST', hospital, meal, r_cap)
        return False
                
    # Allocate the restaurant to the hospital(s)
    print('debiting r capacity: ', r_code, rest_cap.loc[r_code, meal])
    rest_cap.loc[r_code, meal] -= target_quantity
            
    print('increasing r usage', r_code, target_quantity)
    rest_usage.loc[r_code, meal] += target_quantity
        
    curr_capacity -= target_quantity
                
    print('updating log', hospital, meal, r_code)
    meal_log.loc[(meal_log['hospital'] == hospital) & (meal_log['meal'] == meal), 'restaurant'] = r_code
    
    return True

# HANDLE VIPS

# TODO Sort this by SIZE DESCENDING
for i, row in get_oustanding_vip_meals().iterrows():
    assigned = assign_hospital_order(row)
    if not assigned:
        meal_log.loc[(meal_log['hospital'] == row['hospital']) & (meal_log['meal'] == row['meal']), 'failed'] = True
    print()
    print() 

In [74]:
meal_log[(meal_log.vip == True) & (meal_log.failed == True)]

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed


In [75]:
hospitals_that_failed_placement = set()

def select_and_assign_hospital():
    next_hospital = meal_log[~meal_log.hospital.isin(list(hospitals_that_failed_placement))].groupby('hospital')[['priority_rank', 'restaurant']].agg({
            'restaurant': 'count',
            'priority_rank': 'min'
        }).sort_values(by=['restaurant', 'priority_rank']).reset_index()
    
    if next_hospital.shape[0] > 0:
        next_hospital = next_hospital.iloc[0]
    else:
        return False
    
    print('Next hospital:', next_hospital)

    # Find potential orders to serve, ordered by quantity
    possible_orders = meal_log[(meal_log['hospital'] == next_hospital['hospital']) & (meal_log['restaurant'].isnull()) & (meal_log['failed'] == False)].sort_values(by='quantity', ascending=False)
    
    print('Possible orders', possible_orders)
    for i, row in possible_orders.iterrows():
        if assign_hospital_order(row):
            return True
        else:
            meal_log.loc[
                (meal_log['hospital'] == row['hospital']) & (meal_log['meal'] == row['meal']), 
                'failed'
            ] = True
            
    hospitals_that_failed_placement.add(next_hospital['hospital'])
    return True
    

while curr_capacity > 0 and len(hospitals_that_failed_placement) < total_hospitals:
    retval = select_and_assign_hospital()
    if retval == False:
        break
    print()
    print()

Next hospital: hospital         Boston Healthcare for the Homeless - Albany St...
restaurant                                                       0
priority_rank                                                    1
Name: 0, dtype: object
Possible orders                                               hospital zone priority_rank  \
152  Boston Healthcare for the Homeless - Albany St...    3             1   
153  Boston Healthcare for the Homeless - Albany St...    3             1   
154  Boston Healthcare for the Homeless - Albany St...    3             1   
155  Boston Healthcare for the Homeless - Albany St...    3             1   
156  Boston Healthcare for the Homeless - Albany St...    3             1   
157  Boston Healthcare for the Homeless - Albany St...    3             1   
158  Boston Healthcare for the Homeless - Albany St...    3             1   
159  Boston Healthcare for the Homeless - Albany St...    3             1   
160  Boston Healthcare for the Homeless - Albany St.

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Next hospital: hospital         Boston Healthcare for the Homeless - Southampt...
restaurant                                                       0
priority_rank                                                    2
Name: 0, dtype: object
Possible orders                                               hospital zone priority_rank  \
228  Boston Healthcare for the Homeless - Southampt...    3             2   
229  Boston Healthcare for the Homeless - Southampt...    3             2   
230  Boston Healthcare for the Homeless - Southampt...    3             2   
231  Boston Healthcare for the Homeless - Southampt...    3             2   
232  Boston Healthcare for the Homeless - Southampt...    3             2   
233  Boston Healthcare for the Homeless - Southampt...    3             2   
234  Boston Healthcare for the Homeless - Southampt...    3             2   
235  Boston Healthcare for the Homeless - Southampt...    3             2   
236  Boston Healthcare for the Homeless - Southampt.

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


 lunch_mon 2 60.0
Checking for restaurants already in cluster Counter({'IK': 1})
Found an existing restaurant with capacity IK 70.0
Returning a restaurant with capacity, working on the same day IK 70.0
debiting r capacity:  IK 70.0
increasing r usage IK 60.0
updating log Bowdoin Street Health Center lunch_mon IK


Next hospital: hospital         Brockton Neighborhood Health Center
restaurant                                         0
priority_rank                                      2
Name: 0, dtype: object
Possible orders                                 hospital zone priority_rank    vip  \
290  Brockton Neighborhood Health Center    7             2  False   
291  Brockton Neighborhood Health Center    7             2  False   
292  Brockton Neighborhood Health Center    7             2  False   
293  Brockton Neighborhood Health Center    7             2  False   
294  Brockton Neighborhood Health Center    7             2  False   

            meal  quantity restaurant    day faile

...

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


                                             hospital zone priority_rank  \
35  Beth Israel Deaconess Medical Center - West Ca...    1             3   
36  Beth Israel Deaconess Medical Center - West Ca...    1             3   

      vip        meal  quantity restaurant  day failed  
35  False  dinner_sat      50.0        NaN  sat  False  
36  False  dinner_sun      50.0        NaN  sun  False  
Assigning Beth Israel Deaconess Medical Center - West Campus dinner_sat
Beth Israel Deaconess Medical Center - West Campus dinner_sat 1 50.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) ['BB' 'SH' 'PG']
Number of candidates with capacity 4
RD
NT
NL
SU
debiting r capacity:  RD 300.0
increasing r usage RD 50.0
updating log Beth Israel Deaconess Medical Center - West Campus dinner_sat RD


Next hospital: hospital         Boston Healthcare for the Homeless - Albany St...
restaurant                       

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)



Assigning Pine Street Inn - Site 2 breakfast_tues
Pine Street Inn - Site 2 breakfast_tues 3 7.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 0
CANNOT SERVE HOSPITAL FOR THEIR REQUEST Pine Street Inn - Site 2 breakfast_tues       priority  breakfast_tues  zone_3
code                                  
PG           1             NaN     0.5
LD           1             NaN     0.5
MM           1             NaN     0.5
CS           1             NaN     0.5
SU           2             NaN     0.5
MD           1             NaN     0.5
SH           2             NaN     0.5
BB           2             NaN     0.5
ME           1             NaN     0.5
RT           1             NaN     0.5
CE           1             NaN     0.5
BC           2             NaN     0.5
FV           1             NaN     0.5
MY           1             NaN     0.5
CL           1     

HH           1             NaN     0.5
Assigning Pine Street Inn Shattuck Shelter breakfast_wed
Pine Street Inn Shattuck Shelter breakfast_wed 2 4.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 0
CANNOT SERVE HOSPITAL FOR THEIR REQUEST Pine Street Inn Shattuck Shelter breakfast_wed       priority  breakfast_wed  zone_2
code                                 
PG           1            NaN     0.5
LD           1            NaN     0.5
MM           1            NaN     0.5
CS           1            NaN     0.5
SU           2            NaN     0.5
MD           1            NaN     0.5
SH           2            NaN     0.5
BB           2            NaN     0.5
ME           1            NaN     0.5
RT           1            NaN     0.5
CE           1            NaN     0.5
BC           2            NaN     0.5
FV           1            NaN     0.5
MY           1

In [76]:
# Remaining capacity
curr_capacity

5849.0

In [77]:
# Fullfilled (this should be == global capacity - curr_capacity)
meal_log[meal_log['restaurant'].notnull()]['quantity'].sum()

14151.0

In [78]:
# Missed
meal_log[meal_log['restaurant'].isnull()]['quantity'].sum()

1001.0

In [79]:
# Create empty results frame
output = hospitals.copy()
output.iloc[:, 4:] = np.nan


# construct the double-wide output dataframe
for i, c in enumerate(hospitals.columns[4:]):
    output.insert((i*2+1)+4, c + '_rest', np.nan)
        
for i, row in meal_log.iterrows():
    output.loc[row['hospital'], row['meal']] = row['quantity']
    if not pd.isnull(row['restaurant']):
        output.loc[row['hospital'], row['meal'] + '_rest'] = row['restaurant']
    else:
        output.loc[row['hospital'], row['meal'] + '_rest'] = 'XXX'


# for i, row in meal_log.iterrows():
#     if not pd.isnull(row['restaurant']):
#         val = str(row['restaurant'] + ' - ' + str(row['quantity']))
#     else:
#         val = str('XXX - ' + str(row['quantity']))
#     output.loc[row['hospital'], row['meal']] = val
    
# for i, row in meal_log.iterrows():
#     if not pd.isnull(row['restaurant']):
#         output.loc[row['hospital'], row['meal']] = row['restaurant']
        
output.to_csv('../../output.csv')


In [80]:
output

,code,priority_rank,region,zone,breakfast_mon,breakfast_mon_rest,breakfast_tues,breakfast_tues_rest,breakfast_wed,breakfast_wed_rest,...,dinner_wed,dinner_wed_rest,dinner_thurs,dinner_thurs_rest,dinner_fri,dinner_fri_rest,dinner_sat,dinner_sat_rest,dinner_sun,dinner_sun_rest
name,,,,,,,,,,,,,,,,,,,,,
Whittier Street Health Center,15,3,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Boston Children's Hospital - ED,18,3,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,60.0,ME,60.0,BB,60.0,BB,NaN,NaN,NaN,NaN
Beth Israel Deaconess Medical Center - East Campus,53,3,Longwood,1,55.0,XXX,NaN,NaN,55.0,XXX,...,140.0,ME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Beth Israel Deaconess Medical Center - West Campus,54,3,Longwood,1,NaN,NaN,80.0,XXX,NaN,NaN,...,100.0,SH,100.0,BB,100.0,BB,50.0,RD,50.0,BB
VA Medical Center - West Roxbury,12,1,Roxbury / Jamaica Plain,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mattapan Community Health Center,9,2,Dorchester/Mattapan,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dot House Health Dorchester,11,2,Dorchester/Mattapan,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Upham's Corner Health Center,13,2,Dorchester/Mattapan,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Codman Square Health Center - Site 1,16,2,Dorchester/Mattapan,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
rest_usage.iloc[:, 3:(rest_usage.shape[1] - num_zones)].sum(axis=1).sort_values(ascending=False).sum()

14151.0

In [82]:
equity = meal_log.groupby(['priority_rank', 'hospital'])[['meal', 'vip', 'restaurant']].agg({
    'restaurant': [('orders_fulfilled', 'count')],
    'meal': [('orders_requested', 'count')],
    'vip': [('vip_orders', 'sum')],
})
equity

restaurant  \
                                                                 orders_fulfilled   
priority_rank hospital                                                              
1             Boston Healthcare for the Homeless - Albany Street               14   
              Boston Healthcare for the Homeless - Engagement...                4   
              Good Samaritan Medical Center                                     7   
              Saint Vincent Hospital                                            4   
              VA Medical Center - West Roxbury                                  7   
2             Boston Healthcare for the Homeless - Southampto...               10   
              Bowdoin Street Health Center                                      3   
              Brockton Neighborhood Health Center                               5   
              Brookside Community Health Center                                 5   
              Cambridge Health Alliance - Cambridge Hospital                    3   
              Cambridge Health Alliance - Everett Hospital                      3   
              Codman Square Health Center - Site 1                              7   
              Codman Square Health Center - Site 2                              7   
              Dimock Center                                                     4   
              Dot House Health Dorchester                                       5   
              Hope House Addiction Services                                     7   
              MGH Chelsea HealthCare Center                                     1   
              Mattapan Community Health Center                                  3   
              Pine Street Inn - Site 1                                         14   
              Pine Street Inn - Site 2                                         14   
              Pine Street Inn Holy Family Shelter                               7   
              Pine Street Inn Shattuck Shelter                                 14   
              Southern Jamaica Plain Health Center (part of B...                5   
              Tufts Medical Center                                             10   
              Tufts Medical Center - Cardiothoracic ICU                        10   
              Upham's Corner Health Center                                      4   
3             BIDMC - Chelsea                                                   5   
              Beth Israel Deaconess Medical Center - East Campus                8   
              Beth Israel Deaconess Medical Center - West Campus               14   
              Boston Children's Hospital - ED                                   5   
              East Boston Neighborhood Health Center                            4   
              Massachusetts General Hospital - PICU                             7   
              North End Waterfront Health Center                                4   
              South Boston Community Health Center                              5   
              South End Community Health Center                                 5   
              Whittier Street Health Center                                     5   
4             Signature Healthcare Brockton Hospital                            2   
5             Bournewood Hospital                                              10   

                                                                             meal  \
                                                                 orders_requested   
priority_rank hospital                                                              
1             Boston Healthcare for the Homeless - Albany Street               14   
              Boston Healthcare for the Homeless - Engagement...                4   
              Good Samaritan Medical Center                                     7   
              Saint Vincent Hospital                                       

In [83]:
equity.to_csv('../../equity.csv')

In [84]:
# meal_log[meal_log['hospital'] == 'Boston Medical Center']

In [85]:
# rest_usage.iloc[:, 3:-num_zones].sum(axis=1).sort_values(ascending=False)

In [86]:
rest_stats = meal_log[['restaurant', 'quantity']].groupby('restaurant').agg({
    'quantity': ['sum', 'count', 'mean', 'min', 'max']
})
rest_stats.columns = ['sum_total', 'count', 'mean', 'min', 'max']
rest_stats = rest_stats.sort_values(by='sum_total', ascending=False)
rest_stats

,sum_total,count,mean,min,max
restaurant,,,,,
FC,1533.0,34,45.088235,4.0,150.0
SH,1259.0,34,37.029412,4.0,175.0
CL,1236.0,21,58.857143,10.0,150.0
LP,1200.0,5,240.000000,240.0,240.0
MY,1061.0,16,66.312500,10.0,150.0
BB,1040.0,15,69.333333,4.0,175.0
PG,982.0,18,54.555556,5.0,175.0
ME,925.0,11,84.090909,40.0,150.0
CE,802.0,21,38.190476,5.0,150.0


In [87]:
rest_stats.to_csv('../../rest_stats.csv')